In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
stage = "stage_2"

In [3]:
experiment_name = 'NONAMED'

In [4]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastai.utils.mem import *
import re
import pydicom
import pdb
import pickle
import torch


In [5]:
!ls models

densenet121-53_train_cpu_window_uint8_sz512_cv0.0386_subdural_loss_fold5_of_5.pth
densenet121-53_train_cpu_window_uint8_sz512_cv0.0399_subdural_loss_fold4_of_5.pth
densenet121-53_train_cpu_window_uint8_sz512_cv0.0403_subdural_loss_fold3_of_5.pth
densenet121-53_train_cpu_window_uint8_sz512_cv0.0406_subdural_loss_fold2_of_5.pth
densenet121-53_train_cpu_window_uint8_sz512_cv0.0413_subdural_loss_fold1_of_5.pth
densenet121_sz512_cv0.0738_weighted_loss_fold4_of_5.pth
densenet121_sz512_cv0.0739_weighted_loss_fold5_of_5.pth
densenet121_sz512_cv0.0743_weighted_loss_fold2_of_5.pth
densenet121_sz512_cv0.0748_weighted_loss_fold3_of_5.pth
densenet121_sz512_cv0.0749_weighted_loss_fold1_of_5.pth
resnet101-53_train_cpu_window_uint8_sz512_cv0.0365_subdural_loss_fold5_of_5.pth
resnet101-53_train_cpu_window_uint8_sz512_cv0.0366_subdural_loss_fold4_of_5.pth
resnet101-53_train_cpu_window_uint8_sz512_cv0.0369_subdural_loss_fold2_of_5.pth
resnet101-53_train_cpu_window_uint8_sz512_cv0.0371_subdural_loss_fold3

In [6]:
arch = 'resnet18'
fold=4
lr = 1e-3

model_fn = None
SZ = 512
n_folds=5
n_epochs = 15
n_tta = 10

#model_fn = 'resnet34_sz512_cv0.0821_weighted_loss_fold1_of_5'

if model_fn is not None:
    model_fn_fold = int(model_fn[-6])-1
    assert model_fn_fold == fold

In [7]:
data_dir = Path('data/unzip')
train_dir = data_dir / f'{stage}_train_images'
test_dir = data_dir / f'{stage}_test_images'

In [8]:
fn_to_study_ix = pickle.load(open(f'data/{stage}_fn_to_study_ix.pickle', 'rb'))
study_ix_to_fn = pickle.load(open(f'data/{stage}_study_ix_to_fn.pickle', 'rb'))
fn_to_labels = pickle.load(  open(f'data/{stage}_train_fn_to_labels.pickle', 'rb'))
study_to_data = pickle.load (open(f'data/{stage}_study_to_data.pickle', 'rb'))


FileNotFoundError: [Errno 2] No such file or directory: 'data/stage_2_fn_to_study_ix.pickle'

In [ ]:
# gets a pathlib.Path, returns labels
labels = [
    'any0', 'epidural0', 'intraparenchymal0', 'intraventricular0', 'subarachnoid0', 'subdural0',
    'any1', 'epidural1', 'intraparenchymal1', 'intraventricular1', 'subarachnoid1', 'subdural1',
    'any2', 'epidural2', 'intraparenchymal2', 'intraventricular2', 'subarachnoid2', 'subdural2',
]

def get_labels(center_p):

    study, center_ix = fn_to_study_ix[center_p.stem]
    total = len(study_ix_to_fn[study])
    
    # Only 1 study has 2 different RescaleIntercept values (0.0 and 1.0) in the same series, so assume they're all the
    # same and do everything in the big tensor
    
    pixels = {}
    img_labels = []
    ixs = [ max(0, min(ix, total-1)) for ix in range(center_ix-1, center_ix+2) ]
    for i, ix in enumerate(ixs):
        fn = study_ix_to_fn[study][ix]
        img_labels += [ x + str(i) for x in fn_to_labels[fn] ]
            
    return img_labels

get_labels(train_dir / 'ID_ac7c8fe8b.dcm')

In [ ]:
sample_submission = pd.read_csv(f'data/unzip/{stage}_sample_submission.csv')

In [ ]:
sample_submission['fn'] = sample_submission.ID.apply(lambda x: '_'.join(x.split('_')[:2]) + '.dcm')

In [ ]:
sample_submission.head()

In [ ]:
test_fns = sample_submission.fn.unique()

In [ ]:
wc = 50
ww = 100

In [ ]:
def open_dicom(p_or_fn): # when called by .from_folder it's a Path; when called from .add_test it's a string
    #pdb.set_trace()
    center_p = Path(p_or_fn)

    study, center_ix = fn_to_study_ix[center_p.stem]
    total = len(study_ix_to_fn[study])
    
    # Only 1 study has 2 different RescaleIntercept values (0.0 and 1.0) in the same series, so assume they're all the
    # same and do everything in the big tensor
    
    pixels = {}
    ixs = [ max(0, min(ix, total-1)) for ix in range(center_ix-1, center_ix+2) ]
    for ix in ixs:
        if ix not in pixels:
            p = center_p.parent / f'{study_ix_to_fn[study][ix]}.dcm'
            dcm = pydicom.dcmread(str(p))
            if ix == center_ix:
                rescale_slope, rescale_intercept = float(dcm.RescaleSlope), float(dcm.RescaleIntercept)
            pixels[ix] = torch.FloatTensor(dcm.pixel_array.astype(np.float))
        
    t = torch.stack([pixels[ix] for ix in ixs], dim=0) # stack chans together
    if (t.shape[1:] != (SZ,SZ)):
        t = torch.nn.functional.interpolate(
            t.unsqueeze_(0), size=SZ, mode='bilinear', align_corners=True).squeeze_(0) # resize
    t = t * rescale_slope + rescale_intercept # rescale
    t = torch.clamp(t, wc-ww/2, wc+ww/2) # window
    t = (t - (wc-ww/2)) / ww # normalize
    
    return Image(t)


class DicomList(ImageList):
    def open(self, fn): return open_dicom(fn)

In [ ]:
my_stats = ([0.45, 0.45, 0.45], [0.225, 0.225, 0.225])

In [ ]:
class OverSamplingCallback(LearnerCallback):
    def __init__(self,learn:Learner):
        super().__init__(learn)
        self.labels = self.learn.data.train_dl.dataset.y.items
        _, counts = np.unique(self.labels,return_counts=True)
        self.weights = torch.DoubleTensor((1/counts)[self.labels])
        self.label_counts = np.bincount([self.learn.data.train_dl.dataset.y[i].data for i in range(len(self.learn.data.train_dl.dataset))])
        self.total_len_oversample = int(self.learn.data.c*np.max(self.label_counts))
        
    def on_train_begin(self, **kwargs):
        self.learn.data.train_dl.dl.batch_sampler = BatchSampler(WeightedRandomSampler(weights,self.total_len_oversample), self.learn.data.train_dl.batch_size,False)

In [ ]:
# From: https://forums.fast.ai/t/is-there-any-built-in-method-to-oversample-minority-classes/46838/5
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset, indices=None, num_samples=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) if indices is None else indices
            
        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = defaultdict(int)
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            label_to_count[label] += 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)] for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        return 'any1' in dataset.y[idx].obj
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

In [ ]:
folds = np.array_split(list({v['fold'] for v in study_to_data.values()}), n_folds)
folds

In [ ]:
df = pd.read_csv(f"data/{stage}_train_dicom_diags_norm.csv")
# split train/val
np.random.seed(666)
studies_in_fold = [k for k,v in study_to_data.items() if np.isin(v['fold'],folds[fold])]
study_id_val_set = set(df[ df['SeriesInstanceUID'].isin(studies_in_fold)]['SOPInstanceUID'] + '.dcm')

def is_val(p_or_fn):
    p = Path(p_or_fn)
    return p.name in study_id_val_set

In [ ]:
tfms = (
    [ flip_lr(p=0.5), rotate(degrees=(-180,180), p=1.) ], # train
    [] # val
)
data = (DicomList.from_folder(f'data/unzip/{stage}_train_images/', extensions=['.dcm'], presort=True)
    .split_by_valid_func(is_val)
    .label_from_func(get_labels, classes=labels)
    .transform(tfms, mode='nearest')
    .add_test(f'data/unzip/{stage}_test_images/' + test_fns))


In [ ]:
db = data.databunch().normalize()

In [ ]:
balsampler = ImbalancedDatasetSampler(db.train_ds)

In [ ]:
db.train_dl.batch_sampler = balsampler
db.train_dl.batch_sampler
db

In [ ]:
db.show_batch() # AttributeError: 'list' object has no attribute 'pixel' <-- because using torch datasets or whatever

In [ ]:
# yuval reina's loss
yuval_weights = FloatTensor([ 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1 ]).cuda() 

def yuval_loss(y_pred,y_true):
    return F.binary_cross_entropy_with_logits(y_pred,
                                  y_true,
                                  yuval_weights.repeat(y_pred.shape[0],1))

In [ ]:
real_lb_weights = FloatTensor([ 2, 1, 1, 1, 1, 1 ])

def real_lb_loss(pred:Tensor, targ:Tensor)->Rank0Tensor:
    pred,targ = flatten_check(pred,targ)
    tp = pred.view(-1,18)[:,6:12]
    tt = targ.view(-1,18)[:,6:12]
    return F.binary_cross_entropy_with_logits(tp, tt, real_lb_weights.to(device=pred.device))

In [ ]:
w_loss = 0.1
weighted_lb_weights = FloatTensor([ 2.*w_loss, 1.*w_loss, 1.*w_loss, 1.*w_loss, 1.*w_loss, 1.*w_loss, 
                                    2.,        1.,        1.,        1.,        1.,        1., 
                                    2.*w_loss, 1.*w_loss, 1.*w_loss, 1.*w_loss, 1.*w_loss, 1.*w_loss, ])

def weighted_loss(pred:Tensor,targ:Tensor)->Rank0Tensor:
    return F.binary_cross_entropy_with_logits(pred,targ,weighted_lb_weights.to(device=pred.device))

In [ ]:
lb_weights = FloatTensor([ 2, 1, 1, 1, 1, 1 ]).cuda()

def lb_loss(pred:Tensor, targ:Tensor)->Rank0Tensor:
    pred,targ = flatten_check(pred,targ)
    tp = pred.view(-1,6)
    tt = targ.view(-1,6)
    return torch.nn.functional.binary_cross_entropy_with_logits(tp, tt, lb_weights.to(device=pred.device))

In [ ]:
learn = cnn_learner(
    db, getattr(models, arch), loss_func=weighted_loss, metrics=[real_lb_loss],
    pretrained=True, lin_ftrs=[],ps=0.,
    model_dir = Path('./models/').resolve())

In [ ]:
try:
    learn.load(model_fn,strict=False)
    print(f"Loaded {model_fn}")
except Exception as e:
    print(e)

In [ ]:
learn = learn.to_parallel().to_fp16()

arch_to_batch_factor = {
    'resnet18' : 4.3,
    'resnet34' : 3.4,
    'resnet50' : 1.5,
    'resnet101' : 1.0,
    'resnext50_32x4d': 1.2,
    'densenet121': 1.1,
    'squeezenet1_0': 2.6,
    'vgg16': 1.5
}

bs = arch_to_batch_factor[arch] * 512*512 * gpu_mem_get()[0] * 1e-3 / (SZ*SZ)
bs *= int(int(any([isinstance(cb, MixedPrecision) for cb in learn.callbacks])))+1 # 2x if fp16
if any([isinstance(cb, fastai.distributed.ParallelTrainer) for cb in learn.callbacks]):
    bs *= torch.cuda.device_count()
    bs = (bs // torch.cuda.device_count()) * torch.cuda.device_count()
bs = int(bs)
learn.data.batch_size = bs
bs

In [ ]:
learn

In [ ]:
learn.unfreeze()
learn.summary()

In [ ]:
try:
    if lr is None:
        !rm {learn.model_dir}/tmp.pth
        learn.lr_find()
        learn.recorder.plot(suggestion=True)
except Exception as e:
    print(e)
# set lr 

In [ ]:
learn.fit_one_cycle(n_epochs, lr)

# Save

In [ ]:
v = learn.validate()
cv = float(v[-1])

In [ ]:
model_fn = f'{arch}_sz{SZ}_cv{cv:0.4f}_{learn.loss_func.__name__}_fold{fold+1}_of_{n_folds}'
learn.save(model_fn)
model_fn

# Predictions

In [ ]:
learn.data.test_ds.tfms = learn.data.train_ds.tfms
test_preds = []
# Fastai WTF: it figures out to run outputs through sigmoid if a standard loss error is used 
# (see loss_func_name2activ and related stuff) but on custom loss funcs sigmoid must be executed explicitly:
# https://forums.fast.ai/t/shouldnt-we-able-to-pass-an-activ-function-to-learner-get-preds/50492
for _ in progress_bar(range(n_tta)):
    preds, _ = learn.get_preds(DatasetType.Test, activ=torch.sigmoid)
    test_preds.append(preds)
tta_test_preds = torch.cat([p.unsqueeze(0) for p in test_preds],dim=0)

In [ ]:
learn.data.valid_ds.tfms = learn.data.train_ds.tfms
valid_preds = []
# Fastai WTF: it figures out to run outputs through sigmoid if a standard loss error is used 
# (see loss_func_name2activ and related stuff) but on custom loss funcs sigmoid must be executed explicitly:
# https://forums.fast.ai/t/shouldnt-we-able-to-pass-an-activ-function-to-learner-get-preds/50492
for _ in progress_bar(range(n_tta)):
    preds, _ = learn.get_preds(DatasetType.Valid, activ=torch.sigmoid)
    valid_preds.append(preds)
tta_valid_preds = torch.cat([p.unsqueeze(0) for p in valid_preds],dim=0)

In [ ]:
PREDS_DIR = 'data/predictions'
!mkdir -p {PREDS_DIR}

In [ ]:
torch.save(tta_test_preds,  f'{PREDS_DIR}/{model_fn}_test.pth')
torch.save(tta_valid_preds, f'{PREDS_DIR}/{model_fn}_valid.pth')

In [ ]:
pd.DataFrame({'fn': [p.stem for p in learn.data.valid_ds.x.items]}).to_csv(
    f'{PREDS_DIR}/{model_fn}_valid_fns.csv',index=False)
pd.DataFrame({'fn': [Path(p).stem for p in learn.data.test_ds.x.items]}).to_csv(
    f'{PREDS_DIR}/{model_fn}_test_fns.csv',index=False)

# Submit to Kaggle

In [ ]:
tta_test_preds_mean = tta_test_preds.mean(dim=0)
tta_test_preds_geomean = torch.expm1(torch.log1p(tta_test_preds).mean(dim=0))

In [ ]:
ids = []
labels = []

for fn, pred in zip(test_fns, tta_test_preds_mean):
    for i, label in enumerate(db.train_ds.classes):
        if label.endswith('1'):
            ids.append(f"{fn.split('.')[0]}_{label.strip('1')}")
            predicted_probability = '{0:1.10f}'.format(pred[i].item())
            labels.append(predicted_probability)

In [ ]:
mkdir -p data/submissions

In [ ]:
sub_name = experiment_name + "_" + model_fn
sub_path = f'data/submissions/{sub_name}.csv.zip'
sub_name, sub_path

In [ ]:
pd.DataFrame({'ID': ids, 'Label': labels}).to_csv(sub_path, compression='zip', index=False)

In [ ]:
!ls data/submissions/

In [ ]:
!echo {sub_path}

In [ ]:
!kaggle competitions submit -c rsna-intracranial-hemorrhage-detection -f {sub_path} -m {model_fn}